## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Ulladulla,AU,2021-08-08 18:15:00,-35.3500,150.4667,46.09,67,92,4.12,overcast clouds
1,1,Hilo,US,2021-08-08 18:14:28,19.7297,-155.0900,76.73,78,90,4.61,overcast clouds
2,2,Kirensk,RU,2021-08-08 18:15:01,57.7853,108.1119,51.39,76,93,1.03,overcast clouds
3,3,Clyde River,CA,2021-08-08 18:15:01,70.4692,-68.5914,47.16,54,28,10.63,scattered clouds
4,4,Lebu,CL,2021-08-08 18:15:02,-37.6167,-73.6500,50.41,65,42,9.28,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Hilo,US,2021-08-08 18:14:28,19.7297,-155.0900,76.73,78,90,4.61,overcast clouds
6,6,Jieshi,CN,2021-08-08 18:15:03,22.8134,115.8257,82.00,91,98,8.70,light rain
7,7,Butaritari,KI,2021-08-08 18:15:03,3.0707,172.7902,82.65,64,21,10.20,few clouds
12,12,Celestun,MX,2021-08-08 18:15:07,20.8667,-90.4000,88.56,60,4,10.60,clear sky
14,14,Cayenne,GF,2021-08-08 18:15:07,4.9333,-52.3333,87.10,78,75,5.01,broken clouds
17,17,Kapaa,US,2021-08-08 18:15:09,22.0752,-159.3190,80.92,68,90,1.01,overcast clouds
25,25,Ammon,US,2021-08-08 18:15:11,43.4696,-111.9666,84.25,27,1,3.00,clear sky
26,26,Albany,US,2021-08-08 18:14:15,42.6001,-73.9662,87.13,61,41,3.51,scattered clouds
29,29,Sao Filipe,CV,2021-08-08 18:15:12,14.8961,-24.4956,80.15,77,19,7.92,few clouds
32,32,Mandera,KE,2021-08-08 18:15:13,3.9366,41.8670,87.37,32,79,11.07,broken clouds


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID        0
City           0
Country        0
Date           0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = preferred_cities_df.dropna()
# There are no empty rows.

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hilo,US,76.73,overcast clouds,19.7297,-155.0900,
6,Jieshi,CN,82.00,light rain,22.8134,115.8257,
7,Butaritari,KI,82.65,few clouds,3.0707,172.7902,
12,Celestun,MX,88.56,clear sky,20.8667,-90.4000,
14,Cayenne,GF,87.10,broken clouds,4.9333,-52.3333,
17,Kapaa,US,80.92,overcast clouds,22.0752,-159.3190,
25,Ammon,US,84.25,clear sky,43.4696,-111.9666,
26,Albany,US,87.13,scattered clouds,42.6001,-73.9662,
29,Sao Filipe,CV,80.15,few clouds,14.8961,-24.4956,
32,Mandera,KE,87.37,broken clouds,3.9366,41.8670,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Hotel found.")
    except (IndexError):
            print("Hotel not found... skipping")
            pass


Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel not found... skipping
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel not found... skipping
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel found.
Hotel not found... skipping
Hotel found.
Hotel found.
Hotel found.
Hotel 

In [13]:
# Confirm number of empty rows for Hotel Name.
hotel_df.isnull().sum()

City           0
Country        0
Max Temp       0
Description    0
Lat            0
Lng            0
Hotel Name     0
dtype: int64

In [14]:
# 7. Drop the rows where there is no Hotel Name.
# Replace empty spaces with nans and then drop the NaNs.
#hotel_df = hotel_df.replace(r'^s*$', float('Nan'), regex=True)
hotel_df = hotel_df.replace(r'^s*$', np.nan, regex=True)

In [15]:
print(hotel_df)

                          City Country  Max Temp       Description      Lat  \
1                         Hilo      US     76.73   overcast clouds  19.7297   
6                       Jieshi      CN     82.00        light rain  22.8134   
7                   Butaritari      KI     82.65        few clouds   3.0707   
12                    Celestun      MX     88.56         clear sky  20.8667   
14                     Cayenne      GF     87.10     broken clouds   4.9333   
..                         ...     ...       ...               ...      ...   
672                     Sangin      AF     88.88         clear sky  32.0728   
678                   Manikpur      IN     81.21   overcast clouds  25.0667   
679  Santa Isabel Do Rio Negro      BR     75.36   overcast clouds  -0.4139   
680                       Aksu      CN     77.13        few clouds  41.1231   
684                      Praia      CV     82.94  scattered clouds  14.9215   

          Lng            Hotel Name  
1   -155.0900

In [16]:
# Confirm number of Nans. 
len(hotel_df) - hotel_df['Hotel Name'].count()

16

In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace=True)

In [18]:
# Confirm number of Nans
len(hotel_df) - hotel_df['Hotel Name'].count()

0

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Hilo,US,76.73,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Jieshi,CN,82.00,light rain,22.8134,115.8257,Yuehai Hotel
7,Butaritari,KI,82.65,few clouds,3.0707,172.7902,Isles Sunset Lodge
12,Celestun,MX,88.56,clear sky,20.8667,-90.4000,Posada Lilia
14,Cayenne,GF,87.10,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi


In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))